<a href="https://colab.research.google.com/github/FischerMoseley/ball_boi/blob/main/BallBotInfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='3e23b46200d113f9432c17b17751e16e7b42c995', drake_version='0.27.0', drake_build='release')

# Setup matplotlib.
from IPython import get_ipython
if get_ipython() is not None: get_ipython().run_line_magic("matplotlib", "inline")

# python libraries
import numpy as np
from IPython.display import HTML, display
from matplotlib import pyplot
import pydrake.all
from pydrake.all import (
    DiagramBuilder, LinearQuadraticRegulator, Saturation, SceneGraph, Simulator, MakeVectorVariable, Variable,
    WrapToSystem, AddMultibodyPlantSceneGraph, Parser, MathematicalProgram,MultibodyPlant, PlanarSceneGraphVisualizer, MultibodyForces_,
    Expression,DecomposeLumpedParameters, VectorSystem, LogOutput,DiscreteAlgebraicRiccatiEquation
)

from underactuated.jupyter import AdvanceToAndVisualize, SetupMatplotlibBackend, running_as_notebook
import numpy as np
from ipywidgets import FloatSlider, ToggleButton
from IPython.display import display, SVG
import pydot

# underactuated imports
from underactuated import FindResource, ManipulatorDynamics

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Cloning into '/opt/underactuated'...

HEAD is now at 3e23b46 Updated SHA and added textbook question (#425)

ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: bokeh 2.3.2 has requirement pillow>=7.1.0, but you'll have pillow 7.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.



So we are going to see if we can generate the graphs we need to,

In [ ]:
#These are the evaulation Functions
def integrated_cost(q, u, goal, T):
  total_distance_from_target = 0
  total_actuation = 0
  goal_x = goal[0]

  for t in range(len(q)):
    cur_x = q[t][0]
    distance = (cur_x**2 + goal_x**2)**0.5
    total_distance_from_target += distance

    total_actuation += u[t][0]  # assuming u_x = u_theta since that's what Jorge said

  return total_distance_from_target/len(q), total_actuation/len(q)
def cost_breakdown(q, u, goal, T, display=False, yscale="linear"):
  total_distance_from_target = 0
  total_actuation = 0
  goal_x = goal[0]

  distance_from_target = np.array([(q[t][0]**2 + goal_x**2)**0.5 for t in range(len(q))])
  actuation = np.array([u[t][0] for t in range(len(q))])  # assuming u_x = u_theta since that's what Jorge said

  distance_squared = distance_from_target**2
  actuation_squared = actuation**2
  cost = distance_squared + actuation_squared
  total_cost = np.sum(cost)

  if display:
    cost_sum = np.zeros(cost.shape)
    total = 0
    for i in range(cost.shape[0]):
      total += cost[i]
      cost_sum[i] = total

    time_steps = len(q)
    increment = T/time_steps
    times = [t*increment for t in range (len(q))]
    plt.plot(times, distance_from_target, label="Distance From Target")
    plt.plot(times, actuation, label="Actuation")
    plt.plot(times, cost, label="Cost")
    plt.plot(times, cost_sum, label="Total Cost")
    plt.yscale(yscale)
    plt.legend()
    plt.show()

  return distance_from_target, actuation, cost, total_cost

def settling_time(q, u, goal, T, epsilon=1e-3):
  """ Return None if never settled """
  settling_time = None
  goal_x = goal[0]

  for t in range(q.shape[0] - 1, -1, -1):
    cur_x = q[t][0]
    if abs(cur_x - goal_x) > epsilon:  # meaning, they're unequal
      settling_time = t + 1
      break

  if abs(q[-1][0] - goal_x) > epsilon:
    return None

  time_steps = len(q)
  increment = T/time_steps
  settling_time *= increment
  return settling_time

def overshoot(q, u, goal, T, epsilon=1e-3, display=False):
  """ Return None if never reached goal """
  settling_time = None
  goal_x = goal[0]
  start_x = q[0][0]
  timestep_reached_goal = None

  if start_x < goal_x:
    for t in range(len(q)):
      cur_x = q[t][0]
      if cur_x + epsilon >= goal_x:
        timestep_reached_goal = t  # overestimate if not exactly equal
        break
  else:
    for t in range(len(q)):
      cur_x = q[t][0]
      if cur_x - epsilon <= goal_x:
        timestep_reached_goal = t  # overestimate if not exactly equal
        break

  total_overshoot = 0
  for t in range(timestep_reached_goal, len(q)):
    cur_x = q[t][0]
    total_overshoot += abs(goal_x - cur_x)

  if display:
    # here, I'll plot the distance from goal as a function of time
    xs = np.array([q[t][0] for t in range(len(q))])
    distances = xs - goal_x
    time_steps = len(q)
    increment = T/time_steps
    times = [t*increment for t in range(len(q))]
    plt.plot(times, distances)
    plt.title("Distance from Goal")
    plt.xlabel("Time (s)")
    plt.ylabel("Distance")
    plt.show()

  return total_overshoot/len(q)  # can def change this

Now we'll define the 3 different Urdifs we'll use

In [1]:
ballbot_standard = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".2" ixy="0" ixz="0" iyy="0.2" iyz="0" izz="0.2" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.1</mechanicalReduction>
  </transmission>
</robot>
"""
#here the ball is 20% heavier and has 20% more moment of inertia
ballbot_heavyball = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="6" />
      <inertia ixx=".24" ixy="0" ixz="0" iyy="0.24" iyz="0" izz="0.24" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.1</mechanicalReduction>
  </transmission>
</robot>
"""

#here the ball is 20% smaller in radius
ballbot_smallball = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".2" ixy="0" ixz="0" iyy="0.2" iyz="0" izz="0.2" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".08" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.1</mechanicalReduction>
  </transmission>
</robot>
"""

Now lets get to the actual FUN part here we'll have a helper function to create the ball bot, then we'll create a few different diagrams to hook it up to different controllers and plot the results!

In [ ]:
def BallBot():
  builder = DiagramBuilder()
  ballbot, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
  Parser(ballbot).AddModelFromString(ballbot_urdf, "urdf")
  ballbot.set_name("ballbot")
  ballbot.Finalize()

  B = np.array([[1],[1]])

  gain = builder.AddSystem(pydrake.systems.primitives.MatrixGain(B))
  gain.set_name("Actuator Mapping")

  builder.Connect(gain.get_output_port(),ballbot.get_actuation_input_port())
  builder.ExportInput(gain.get_input_port(),"torque")
  builder.ExportOutput(ballbot.get_state_output_port(), "continuous_state")
  builder.ExportOutput(scene_graph.get_query_output_port(), "query")
  builder.ExportOutput(scene_graph.get_pose_bundle_output_port(), "pose")

  return builder.Build(),scene_graph
